# Import packages

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 500)

from datetime import datetime
import pickle

import ta
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import webbrowser
from datetime import datetime, timedelta

import random

plt.style.use('classic')
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_rows', 300)

# %config inlinebackend.figure_format = 'svg'

In [2]:
import sys

sys.path.insert(0, 'D:/Intraday_trading')

In [3]:
from src.support import *
from src.backtest import *
from src.models import *
from src.models_support import *

# Import and pre-processing data

## Interval data

In [4]:
%%time

interval_data = pd.read_pickle('D:/Intraday_trading/Training/Data/XAUUSD_M15_1.pkl')

interval_data = interval_data.set_index('DATE_TIME')
interval_data.index = pd.to_datetime(interval_data.index)

interval_data['DATE'] = pd.to_datetime(interval_data['DATE'])
interval_data['OPEN'] = interval_data['OPEN']
interval_data['HIGH'] = interval_data['HIGH']
interval_data['LOW'] = interval_data['LOW']
interval_data['CLOSE'] = interval_data['CLOSE']


CPU times: total: 15.6 ms
Wall time: 48 ms


In [5]:
interval_data.shape

(99741, 8)

In [6]:
interval_data.index[0], interval_data.index[-1] 

(Timestamp('2021-01-03 00:00:00'), Timestamp('2025-03-21 20:45:00'))

### Prepare_df

In [7]:
df_4_hour = prepare_df(df = interval_data, timeframe = '4H', add_indicators = True)
df_1_day = prepare_df(df = interval_data, timeframe = '1D', add_indicators = True)
df_15_min = prepare_df(df = interval_data, timeframe = '15min', add_indicators = True)

df_1_day['WHOLE_RANGE'] = df_1_day['HIGH'] - df_1_day['LOW']
df_4_hour['WHOLE_RANGE'] = df_4_hour['HIGH'] - df_4_hour['LOW']
df_15_min['WHOLE_RANGE'] = df_15_min['HIGH'] - df_15_min['LOW']

df_1_day['GRP_WHOLE_RANGE'] = pd.qcut(df_1_day['WHOLE_RANGE'], 10)
df_4_hour['GRP_WHOLE_RANGE'] = pd.qcut(df_4_hour['WHOLE_RANGE'], 10)
df_15_min['GRP_WHOLE_RANGE'] = pd.qcut(df_15_min['WHOLE_RANGE'], 10)

df_1_day['GRP_BODY'] = pd.qcut(df_1_day['BODY'], 10)
df_4_hour['GRP_BODY'] = pd.qcut(df_4_hour['BODY'], 10)
df_15_min['GRP_BODY'] = pd.qcut(df_15_min['BODY'], 10)

df_1_day['YEAR'] = df_1_day.index.strftime('%Y')
df_1_day['WEEK'] = df_1_day.index.strftime('%Y%W')
df_1_day['MONTH'] = df_1_day.index.strftime('%Y%m')

df_4_hour['YEAR'] = df_4_hour.index.strftime('%Y')
df_4_hour['WEEK'] = df_4_hour.index.strftime('%Y%W')
df_4_hour['MONTH'] = df_4_hour.index.strftime('%Y%m')

df_15_min['YEAR'] = df_15_min.index.strftime('%Y')
df_15_min['WEEK'] = df_15_min.index.strftime('%Y%W')
df_15_min['MONTH'] = df_15_min.index.strftime('%Y%m')



In [8]:
df_15_min.shape, df_4_hour.shape, df_1_day.shape, 

((99741, 36), (6783, 36), (1311, 36))

In [9]:
df_1_day.columns

Index(['OPEN', 'HIGH', 'LOW', 'CLOSE', 'TICK_VOL', 'AVG_PRICE',
       'FLAG_INCREASE_CANDLE', 'BODY', 'UPPER_SHADOW', 'LOWER_SHADOW',
       'WHOLE_RANGE', 'FLAG_LONG_UPPER_SHADOW', 'FLAG_LONG_LOWER_SHADOW',
       'FLAG_HIGHER_HIGH(20)', 'FLAG_HIGHER_LOW(20)', 'AVG_VOL(50)',
       'FLAG_OVER_AVG_VOL(50)', 'AVG_VOL(200)', 'FLAG_OVER_AVG_VOL(200)',
       'FLAG_UPTREND_VOL(20)', 'RSI', 'FLAG_UNDER_30_RSI', 'FLAG_OVER_70_RSI',
       'FLAG_UPTREND_RSI(20)', 'BB_UPPER_BAND(50)', 'BB_LOWER_BAND(50)',
       'EMA(50)', 'POSITION_EMA(50)', 'EMA(200)', 'POSITION_EMA(200)',
       'Ret(t)', 'GRP_WHOLE_RANGE', 'GRP_BODY', 'YEAR', 'WEEK', 'MONTH'],
      dtype='object')

In [10]:
# df_15_min.index[0], df_15_min.index[-1]

In [11]:
# plot_df(df_1_day, 
#         path = None,# 'D:/Intraday_trading/Training/Saved_results/plot_df.html', 
#         open_tab = False)

# Gamma_v1

In [12]:
df_1_day_observed = df_1_day.loc[(df_1_day.index <= datetime(2023, 3, 7, 0)), :].tail(40).copy()
df_4_hour_observed = df_4_hour.loc[(df_4_hour.index <= datetime(2023, 3, 7, 0)), :].tail(40).copy()
df_15_min_observed = df_15_min.loc[(df_15_min.index <= datetime(2023, 3, 7, 0)), :].tail(40).copy()


In [13]:
plot_df(df_1_day_observed, 
        path = None,# 'D:/Intraday_trading/Training/Saved_results/plot_df.html', 
        open_tab = False)

In [14]:
plot_df(df_4_hour_observed, 
        path = None,# 'D:/Intraday_trading/Training/Saved_results/plot_df.html', 
        open_tab = False)

In [15]:
plot_df(df_15_min_observed, 
        path = None,# 'D:/Intraday_trading/Training/Saved_results/plot_df.html', 
        open_tab = False)

In [16]:
def _cal_trend_score(df_observe):
    '''
    Identify trend and reversal based on 
    - previous highs and lows
    - breaks of previous highs and lows
    - max changes (in percentage or point) in period
    - current changes (in percentage or point) in period
    '''
    
    df_highlow = swing_highs_lows(df_observe, swing_length = 3)
    df_highlow = df_highlow[df_highlow['HIGHLOW'].isnull() == False]

    df_bos_choch = bos_choch(df_observe, swing_highs_lows(df_observe, swing_length = 3), close_break = True)
    df_bos_choch = df_bos_choch[((df_bos_choch['BOS'].isnull() == False) | (df_bos_choch['CHOCH'].isnull() == False))]

    current_period_change = df_observe['CLOSE'][-1] - df_observe['CLOSE'][0]
    max_period_change = df_observe['HIGH'].max() - df_observe['LOW'].min()

    local_max = df_highlow[df_highlow['HIGHLOW'] == 1]
    local_min = df_highlow[df_highlow['HIGHLOW'] == -1]

    bos_up = df_bos_choch[df_bos_choch['BOS'] == 1]
    bos_down = df_bos_choch[df_bos_choch['BOS'] == -1]
    
    choch_up = df_bos_choch[df_bos_choch['CHOCH'] == 1]
    choch_down = df_bos_choch[df_bos_choch['CHOCH'] == -1]

    trend_score = (np.abs(current_period_change) > 0)*np.sign(current_period_change) + \
                (np.abs(current_period_change) > 10)*2*np.sign(current_period_change) + \
                (np.abs(current_period_change) > 15)*3*np.sign(current_period_change) + \
                (np.abs(current_period_change/max_period_change) > 0.5)*np.sign(current_period_change) + \
                (np.abs(current_period_change/max_period_change) > 0.7)*2*np.sign(current_period_change) + \
                (np.abs(current_period_change/max_period_change) > 0.9)*3*np.sign(current_period_change) + \
                (len(local_max) <= 5)*np.sign(current_period_change) + \
                (len(local_min) <= 5)*np.sign(current_period_change) + \
                (len(local_max) < 3)*2*np.sign(current_period_change) + \
                (len(local_min) < 3)*2*np.sign(current_period_change) + \
                (len(bos_up) > len(bos_down))*(current_period_change > 0) + \
                (len(bos_up) < len(bos_down))*(current_period_change < 0)*-1 + \
                (len(bos_up) > len(bos_down) + 2)*2*(current_period_change > 0) + \
                (len(bos_up) < len(bos_down) + 2)*2*(current_period_change < 0)*-1 + \
                (len(choch_up) > len(choch_down))*(current_period_change < 0)*2 + \
                (len(choch_up) < len(choch_down))*(current_period_change > 0)*-2

    trend_score /= 22

    return(trend_score)

In [17]:
_cal_trend_score(df_1_day_observed), _cal_trend_score(df_4_hour_observed), _cal_trend_score(df_15_min_observed)

(-0.5454545454545454, 0.45454545454545453, -0.4090909090909091)

In [19]:
(_cal_trend_score(df_1_day_observed)*1 + _cal_trend_score(df_4_hour_observed)*1 + _cal_trend_score(df_15_min_observed)*8)/10

-0.33636363636363636

In [ ]:

class Base_Alpha:
    def __init__(self, strat, prepare_data):
        # Prepare function
        self.prepare_data = prepare_data
        # Strategy function
        self.strat = strat 

        self.config = None

    def signal(self, df_observe):
        '''
        Return the tested dataframe with additional columns: signal (buy/ sell), stop loss, take profit.
        In case eval = False --> live trading, return only the vector of (signal, stop_loss, take_profit).
        '''
        df_prepare = self.prepare_data(df_observe)           
        # Apply the strategy to the dataframe and return the result in IS
        df_result = df_prepare.copy().apply(self.strat, axis=1)

        return(df_result)



class RSI_strat(Base_Alpha):
    def __init__(self, config):
        super().__init__(
            strat = self._RSI_strat,
            prepare_data = self._prepare_data
        )  # Pass the strategy function to the parent constructor
        
        '''
        config = {'RSI_param': 14, 'base_SL': 10, 'base_TP': 20}
        '''
        self.RSI_param = config['RSI_param']
        self.base_SL = config['base_SL']
        self.base_TP = config['base_TP']
        
        
    def _prepare_data(self, df_observe):
        df_observe['RSI'] = ta.momentum.RSIIndicator(df_observe['CLOSE'], window = self.RSI_param).rsi()
        return(df_observe)

    def _RSI_strat(self, row):
        '''Strategy for RSI, calculates the signal based on the RSI value.'''
        if row['RSI'] < 30:
            return pd.Series([1, self.base_SL, self.base_TP], index=['SIGNAL', 'SL', 'TP']) # Buy signal
        elif row['RSI'] > 70:
            return pd.Series([-1, self.base_SL, self.base_TP], index=['SIGNAL', 'SL', 'TP']) # Sell signal
        else:
            return pd.Series([0, 0, 0], index=['SIGNAL', 'SL', 'TP']) # No action (neutral)
        
